# Pre-processing of raw SORTseq files (QC, filtering)
<span style="font-size:0.85em;">Written by: Peter Brazda (20 Feb 2024) - Last Update: Farid Keramati, Peter Brazda (13 March 2024)</span>

This workflow aims to help an initial, per-plate (visual) inspection (1) and to prepare expression matrices for the downstream steps (2):
<details><summary><b>1. QC </b></summary>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.1 Set the environment</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.2 Collect the input files</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;1.3 Run the loop to generate the .pdf reports</details>
<details><summary><b>2. Filtering </b></summary>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2.1 Set the environment</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2.2 Collect the input files</br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;2.3 Run the loop to generate the filtered matrices</br>

</details>

 <div class="alert alert-block alert-warning">
&#9888;<b>Warning:</b>Before running the script make sure to download <a><b>Functions_QC.R</b></a> for the helper functions, <a><b>hg38_genes_with_ERCC_unique_ensembl_gene.tab</b></a> for the genome annotation and <a><b>Plate_distribution_simple.xlsx</b></a> for the 384-well plate format and the raw matrix files for example <a><b>TM621.transcripts</b></a> (the format of the name is important). </br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The pipeline requires an output folder for the compiled .pdf reports and also a 'temp' folder for the intermediate .png files.
</div>

## 1. Generate QC-reports per plate

### 1.1 Set the environment
Identify the input/output folders and load the helper files.

In [8]:

home <- "" # select home directory

setwd(home)

source(paste0(home,"/helper_files/Functions_QC.R")) # the function to generate the QC plots for one plate
dir <-  paste0(home,"/out/qc_plots/") # location for the final output .pdf file (the per-plate visual report)
temp <- paste0(home,"/out/qc_plots/temp") # location for the temporary .png files
matrices <- paste0(home,"/in/") # location for the raw matrix files
gencode <- read.table(paste0(home,"/helper_files/hg38_genes_with_ERCC_unique_ensembl_gene.tab"), header = TRUE) # the hg38 reference extended with the ERCC genes
primer <- read_excel(paste0(home,"/helper_files/Plate_distribution_simple.xlsx"), col_names = FALSE) # primer plate distribution file


New names:
* `` -> ...1
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* ...



In [9]:
ls()

[1] "dir"                       "Filter_SORTseq_tables"    
[3] "gencode"                   "home"                     
[5] "matrices"                  "Prepare_SORTseq_QC_report"
[7] "primer"                    "temp"

### 1.2 Collect a list of all .transcript.txt files

In [11]:
transcript_files <- list.files(matrices, '\\.transcripts.txt$', full.names = TRUE)

### 1.3 Loop through each file to generate QC report
It is advised to remove the <code><a>suppressWarnings</a></code> during the first run.

In [ ]:
suppressWarnings({

    for(file_path in transcript_files) {
  setwd(home)
  plateID <- gsub("\\.transcripts\\.txt$", "", basename(file_path)) # extract the plateID from the file name
  setwd(temp)
  
  # Generate QC report for the current plate
  Prepare_SORTseq_QC_report(plateID = plateID, matrices = matrices, primer = primer, gencode = gencode) # the function is defined in Functions_QC.R
  
  # Grab the files from the temp folder
  files <- list.files(temp, '\\.png$', full.names = TRUE)
  rl <- lapply(files, png::readPNG)
  gl <- lapply(rl, grid::rasterGrob)
  
  pdf_file <- paste0(dir, "QCreport_", plateID, ".pdf")
  ggsave(pdf_file, gridExtra::marrangeGrob(grobs = gl, nrow = 5, ncol = 4, top = NULL), width = 8, height = 10)
  file.remove(files) # Remove temporary .png files after saving the report
}
    })
    

## 2. Filter the matrices

### 2.1 Set the environment
Identify the input/output folders and load the helper files.

In [17]:
home <- "" # select home directory


setwd(home)
source(paste0(home,"/helper_files/Functions_QC.R")) # the function to generate the QC plots for one plate
dir <-  paste0(home,"/out/qc_plots/") # location for the final output .pdf file (the per-plate visual report)

gencode <- read.table(paste0(home,"/helper_files/hg38_genes_with_ERCC_unique_ensembl_gene.tab"), header = TRUE) # the hg38 reference extended with the ERCC genes
input_location <- paste0(home,"/in/") # location for the raw matrix files
output_location_nME <- paste0(home,"/out/qc_plots/1_noMITO_noERCC/") # output folder for the matrix after filtering and subsequent removal of mitochondrial AND ERCC genes
output_location_nE <- paste0(home,"/out/qc_plots/2_noERCC/")  # output folder for the matrix after filtering and subsequent removal of ERCC genes



### 2.2 Collect a list of all .transcript.txt files

In [18]:
transcript_files <- list.files(input_location, pattern = "\\.transcripts\\.txt$", full.names = FALSE)

### 2.3 Loop through each file to process them
This loop uses a custom function that identifies mitochondrial and ERCC genes and generates matrices without these genes.
It is advised to remove the <code><a>suppressWarnings</a></code> during the first run.

In [20]:
suppressWarnings({
    for(file_name in transcript_files) {
  plateID <- gsub("\\.transcripts\\.txt$", "", file_name) # extract the plateID from the file name
  
  # Call the filtering function for each plate
  Filter_SORTseq_tables(plate = plateID,  # the function is defined in Functions_QC.R
                        input_location = input_location,
                        # mito.pct.max = mito.pct.max, # set the upper limit for % mitochondrial gene counts per cell, default = 30
                        # ercc.content.max = ercc.content.max # set the upper limit for % ERCC gene counts per cell (high % reflects failed mRNA-capture, but successful amplification), default = 25
                        output_location_nME = output_location_nME,
                        output_location_nE = output_location_nE) 
}
    })


In [21]:
print(sessionInfo())

R version 4.0.5 (2021-03-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 20.04.2 LTS

Matrix products: default
BLAS/LAPACK: /usr/lib/x86_64-linux-gnu/openblas-pthread/libopenblasp-r0.3.8.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=C             
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] viridis_0.6.1     viridisLite_0.4.0 gridExtra_2.3     png_0.1-7        
 [5] plyr_1.8.6        scales_1.1.1      platetools_0.1.5  readxl_1.3.1     
 [9] data.table_1.14.0 forcats_0.5.1     stringr_1.4.0     dplyr_1.0.6      
[13] purrr_0.3.4       readr_1.4.0       tidyr_1.1.3       tibble_3.1.2   